# 6. Sinusoidal Forcing Of A Spring-Mass-Damper System


This notebook introduces external forcing of a vibratory system, where the
external force is modeled as a sinusoidal input to the bottom of a bus driver's
seat.

After the completion of this assignment students will be able to:

- excite a system with a sinusoidal input
- understand the difference in transient and steady state solutions
- use autocorrelation to determine period
- relate the frequency response to the time series
- create a frequency response plot
- define resonance and determine the parameters that cause resonance

# Introduction

We have only been dealing with the free response of a system given some initial coordinate (position, orientation) or speed value (velocity, angular velocity). The behavior observed is the "natural" behavior of the system, i.e. the motion that it would exhibit if no external loads act on the system. For the simple single degree of freedom systems we have dealt with so far it is possible to apply a time varying external force or torque to the system to influence the time trajectory of the single coordinate. For example let's investigate the simplest system in vibrations, a particle that can slide laterally but is fixed to a wall via a linear spring and linear damper and has a sinusoidal force applied to it as it moves:

![](fig/06/Mass-Spring-Damper.svg)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
from resonance.linear_systems import MassSpringDamperSystem

In [ ]:
sys = MassSpringDamperSystem()

The mass, $m$, stiffness of the spring, $k$, and the damping coefficient of the dashpot, $c$, can be set on this model.

In [ ]:
sys.constants

# Undamped Forced Motion

Each system has a function called `sinusoidal_forcing_response()` that works in a similar fashion to `free_response()`. Two new pieces of information are required: the forcing amplitude, $F_o$, and the forcing frequency, $\omega$ as the first two arguments. Set $F_o=.0$ N and $\omega=2\pi$ rad/s and generate a trajectory for 20 seconds.

In [ ]:
traj = sys.sinusoidal_forcing_response(1.0, 2 * np.pi, 20.0)

Notice that there is no longer a simple sinusoidal oscillation. The trajactory seems that it may still be *periodic* but the motion is now some combination of the natural motion and the motion due to the forcing term. 

In [ ]:
traj.plot(subplots=True);

In [ ]:
fig, ax = plt.subplots(1, 1)
traj = sys.sinusoidal_forcing_response(0.0, 0.0, 20.0)
lines = ax.plot(traj.index, traj.position)
ax.set_xlabel('Time [s]')
ax.set_ylabel('Position [m]')
ax.set_ylim((-0.5, 0.5))

def adjust_forcing(amplitude=0.0, frequency=0.0):
    traj = sys.sinusoidal_forcing_response(amplitude, frequency, 20.0)
    lines[0].set_data(traj.index, traj.position)

In [ ]:
from ipywidgets import interact

interact(adjust_forcing, amplitude=(0.0, 10.0, 0.2), frequency=(0.0, 10 * np.pi, 0.3));

# Forcing at and around the natural frequency

You may have notice some particularly interesting motion in a particular regime of frequencies. It turns out that if you force the system at a frequency near the natural frequency of the system. Simulate the free response of the system and find the natural frequency in radians per second. Investigate the forced trajectory with a 1 Newton forcing amplitude and a forcing frequency slightly less than, slightly more than, and exactly at the natural frequency.

In [ ]:
sys.coordinates['position'] = 0.1  # m
traj = sys.free_response(5.0)
traj.plot(subplots=True);

**Exercise**

Estimate the natural frequency.

In [ ]:
from resonance.functions import estimate_period
2 * np.pi / estimate_period(traj.index, traj.position)

# Just lower than the natural frequency

In [ ]:
sys.coordinates['position'] = 0.0  # m

traj = sys.sinusoidal_forcing_response(1.0, 9.8, 40.0)

In [ ]:
traj.plot(subplots=True);

# Just higher than the natural frequency

In [ ]:
traj = sys.sinusoidal_forcing_response(1.0, 10.2, 40.0)
traj.plot(subplots=True);

This is called *beating*. Beating signals seem to have two primary frequencies, one that is much slower than the other. There is a rapid frequency that is close to the forced frequency and then another that can be estimated with:

$$\omega_{beat} = |\omega_n - \omega|$$

Calculate the beat frequency and beat period from the above equation and see if it matches the trajectory in the above plot.

In [ ]:
w_beat = np.abs(9.9959766250584341 - 10.2)
print(w_beat)

print(2 * np.pi / w_beat)

# Exactly at the natural frequency

In [ ]:
traj = sys.sinusoidal_forcing_response(1.0, 9.9959766250584341, 100.0)

In [ ]:
traj.plot(subplots=True);

# Damping

There is also interesting vibrational behavior when both damping and forcing are introduced to the system. Set $m=100$ kg, $c=100$ kg/s, and $k=910$ N/m and simulate the system's free response for 20 seconds to remind yourself of the behavior. Use an initial position of 0.001 m and initial velocity of 0.02 m/s. What type of vibration is this? (over-, under-, critically-damped, etc)

In [ ]:
sys.constants['mass'] = 100  # kg
sys.constants['damping'] = 100  # kg/s
sys.constants['stiffness'] = 910  # N/m

sys.coordinates['position'] = 0.001  # m
sys.speeds['velocity'] = 0.02  # m/s

In [ ]:
traj = sys.free_response(20)
traj.plot(subplots=True);

# Natural Frequency

The exact value of a linear system's natural frequency can be calculate with:

$$\omega_n = \sqrt{\frac{k}{m}}$$

where $m$ is the system mass and $k$ is the system's stiffness. Calculate this value using the equation above.

In [ ]:
np.sqrt(sys.constants['stiffness'] / sys.constants['mass'])

In [ ]:
# write your answer here

# Forcing a damped system below it's natural frequency

Subject the system to $F(t) = 10 \cos(1t)$ with the same initial conditions as the previous case. This frequency is well below the natural frequency. Simulate for 5, 10, 20, and 50 seconds.

In [ ]:
traj = sys.sinusoidal_forcing_response(10.0, 1.0, 5)
traj.plot(subplots=True);

# Just below the natural frequency

Subject the system to $F(t) = 10 \cos(3t)$ with the same initial conditions as the previous case. This frequency is well below the natural frequency. Simulate for 5, 10, 20, and 50 seconds.

In [ ]:
traj = sys.sinusoidal_forcing_response(10.0, 3.0, 5)
traj.plot(subplots=True);

# Higher than natural frequency

Subject the system to $F(t) = 10 \cos(10t)$ with the same initial conditions as the previous case. This frequency is well below the natural frequency. Simulate for 5, 10, 20, and 50 seconds.

In [ ]:
traj = sys.sinusoidal_forcing_response(10.0, 10.0, 5)
traj.plot(subplots=True);

**Excersise**

What do you observe about how the vibration behavior changes with respect to the forcing amplitude and frequency? How does the amplitude and frequency of the position trajectory change with respect to the forcing function? Create a interactive plot where you can adjust the forcing amplitude and frequency and observe the change in position over a 30 second simulation.

In [ ]:
# write your answer here

# Transient and Steady State Behavior

When you increase the simulation time long enough in all of the above forced responses, you should see that there is more unpredictable behavior at the beginning of the response and that in the later portion of the response the system gets into a very predictable sinusoidal motion. What is the frequency of the motion after about 10 seconds of simulation? Do you recognize this frequency?

In [ ]:
# write answer here

The underdamped regime of motion for this system occurs with a damping coefficient value as about $60.33 < c < 603.32$ kg/s.

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
def cosine_func(times, amp, freq, phase_angle):
    return amp * np.cos(freq * times - phase_angle)

**Exercise**

Using the curve fit tool and a fitting function that looks like $X\cos(\omega t - \theta)$ to fit the steady state behavior of the position trajectory ($t > 10$s or so, use `traj[m:]` to get just the last `m` seconds). With the damping ratio set at 100 kg/s make a plot of the position amplitude as a function of forcing frequences $1.0 < \omega < 10.0$ rad/s. Use a forcing amplitude of 10 N. Plot a vertical line using `ax.axvline()`.

```python
Fo = 10.0  # N

frequencies = np.linspace(1.0, 10.0, num=100)
   
sys.constants['damping'] = 100  # kg/s
    
amplitudes = []
    
for omega in frequencies:
    # write the code for the loop here

fig, ax = plt.subplots(1, 1, sharex=True)
ax.set_xlabel('$\omega$ [rad/s]')
ax.set_ylabel('Steady state amplitude [m]') 

# write the plot commands here
```

In [ ]:
Fo = 10.0  # N

frequencies = np.linspace(1.0, 10.0, num=100)
   
sys.constants['damping'] = 100  # kg/s
    
amplitudes = []
    
for omega in frequencies:
    traj = sys.sinusoidal_forcing_response(Fo, omega, 20.0)
    popt, pcov = curve_fit(cosine_func,
                           traj[10:].index, traj[10:].position,
                           p0=(Fo, omega, 0.05))
    amplitudes.append(abs(popt[0]))

amplitudes = np.array(amplitudes)

fig, ax = plt.subplots(1, 1, sharex=True)
ax.set_xlabel('$\omega$ [rad/s]')
ax.set_ylabel('Steady state amplitude [m]') 

ax.axvline(np.sqrt(sys.constants['stiffness'] / sys.constants['mass']), color='black')
ax.plot(frequencies, amplitudes);

**Exercise**

Now let's plot a new line on the plot for different values of damping coefficients, instead of a single value. You can use a *nested loop* to cycle through frequencies for each damping ratio. Use 5 equally spaced values of $60 < c < 600$ and the same parameters as above. Add a legend to show which color line corresponds to what damping coefficient.

```python
Fo = 10.0  # N

dampings = np.linspace(60.0, 600.0, num=5)
frequencies = np.linspace(1.0, 10.0, num=100)
results = []

for c in dampings:
    
    # set the damping
    
    amplitudes = []
    
    for omega in frequencies:
        # write this portion
    
    amplitudes = np.array(amplitudes)
    
    # store the amplitudes in the results list

fig, ax = plt.subplots(1, 1, sharex=True)
ax.set_xlabel('$\omega$ [rad/s]')
ax.set_ylabel('Steady state amplitude [m]') 

ax.axvline(np.sqrt(sys.constants['stiffness'] / sys.constants['mass']), color='black')

for amps in results:
    ax.plot(frequencies, amps)

# add legend
```

In [ ]:
Fo = 10.0  # N

dampings = np.linspace(60.0, 600.0, num=5)
frequencies = np.linspace(1.0, 10.0, num=100)
results = []

for c in dampings:
    
    sys.constants['damping'] = c
    
    amplitudes = []
    
    for omega in frequencies:
        traj = sys.sinusoidal_forcing_response(Fo, omega, 20.0)
        popt, pcov = curve_fit(cosine_func,
                               traj[10:].index, traj[10:].position,
                               p0=(Fo, omega, 0.05))
        amplitudes.append(np.abs(popt[0]))
    
    amplitudes = np.array(amplitudes)
    
    results.append(amplitudes)

fig, ax = plt.subplots(1, 1, sharex=True)
ax.set_xlabel('$\omega$ [rad/s]')
ax.set_ylabel('Steady state amplitude [m]') 

ax.axvline(np.sqrt(sys.constants['stiffness'] / sys.constants['mass']), color='black')

for amps in results:
    ax.plot(frequencies, amps)

ax.legend(dampings);

# The frequency response

The plot you created above is called a *frequency response plot* it shows how the magnitude of the steady state response to a sinusoidal forcing. The X axis is the forcing frequency and the Y axis is the amplitude of the position oscillation. The frequency response plot also typically includes a plot of the position's steady state phase shift angle as a function of the forcing frequency too. 

In [ ]:
sys.constants['damping'] = 60
sys.frequency_response_plot(10.0);

**Exerice**

Create an interactive plot using the `sys.frequency_response_plot()` function with a slider for the damping coefficient that goes from $60 < c < 600$ with an appropriate interval spacing. You can access the lines in the plot like so:

In [ ]:
axes = sys.frequency_response_plot(10.0)
amp_line = axes[0].lines[1]
phase_line = axes[1].lines[1]

In [ ]:
amp_line.get_xdata()

# Extra: Phase Shift and Correlation

In [ ]:
num_samples = 3000
t = np.linspace(0, 10, num=num_samples)
w = 3.0
x1 = np.cos(w * t)
x2 = np.cos(w * t - np.pi / 6)
plt.figure()
plt.plot(t, x1, '.', t, x2, '.')
plt.legend(['cos', 'cos shifted'])

In [ ]:
from scipy.signal import correlate

In [ ]:
plt.figure()
plt.plot(np.arange(1-num_samples, num_samples), correlate(x1, x2))

In [ ]:
xcorr = correlate(x1, x2)
dn = np.arange(1-num_samples, num_samples)
dn[xcorr.argmax()]
t[abs(dn[xcorr.argmax()])] * w

In [ ]:
traj = sys.sinusoidal_forcing_response(10.0, 3.4, 20.0, sample_rate=500)
popt, pcov = curve_fit(cosine_func,
                       traj[10:].index, traj[10:].position,
                       p0=(10.0, 3.4, 0.05))
plt.figure()
plt.plot(traj.index, traj.position, '.',
         traj.index, cosine_func(traj.index, *popt))

In [ ]:
popt

In [ ]:
plt.figure()
plt.plot(traj[10:].index, traj[10:].position / np.max(traj[10:].position))
plt.plot(traj[10:].index, np.cos(3.4*traj[:10].index - popt[-1]))

In [ ]:
def phase_shift(x1, x2, w, dt):
    num_samples = len(x1)
    xcorr = correlate(x1, x2)
    dn = np.arange(1-num_samples, num_samples)
    return dt * abs(dn[xcorr.argmax()]) * w

In [ ]:
phase_shift(traj[10:].position / np.max(traj[10:].position),
            np.cos(3.4*traj[:10].index - popt[-1]),
            3.4,
            1/500)

In [ ]:
t = np.linspace(0.0, 100.0, num=100000)
phase_shift(np.cos(10*t), np.cos(10*t - 0.1), 10.0, 1/100000)

In [ ]:
np.pi / 6

In [ ]:
Fo = 10.0  # N

dampings = np.linspace(60.0, 600.0, num=5)
frequencies = np.linspace(1.0, 10.0, num=100)
results = []

for c in dampings:
    
    sys.constants['damping'] = c
    
    amplitudes = []
    phases = []
    
    for omega in frequencies:
        traj = sys.sinusoidal_forcing_response(Fo, omega, 20.0, sample_rate=400)
        popt, pcov = curve_fit(cosine_func,
                               traj[10:].index, traj[10:].position,
                               p0=(Fo, omega, 0.05))
        amplitudes.append(popt[0])
        theta = phase_shift(np.cos(omega*t), traj.position, omega, 1/400)
        phases.append(theta)
    
    amplitudes = np.array(amplitudes)
    phases = np.array(phases)
    
    results.append([amplitudes, phases])

In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True)
axes[1].set_xlabel('$\omega$ [rad/s]')
axes[0].set_ylabel('Steady state amplitude [m]') 

for amps, shifts in results:
    axes[0].plot(frequencies, amps)
    axes[1].plot(frequencies, shifts)

ax.legend(dampings)